# Import libraries, set options, connect to DB

In [1]:
# Configuration code for datawrangling
import pandas as pd
import os
import numpy as np
from datetime import datetime
from geocode import geocode
import mapToPoly
from mapToPoly import mapToPoly
pd.set_option('display.max_row', 30000)
import csv

# Configuration code in order to connect to the database
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker
from database_setup import Itenerary, Base

passWord = os.environ['my_password']
DATABASE_URI = 'postgres://maxcarey:' + passWord + '@totago.cqfm37jhmjmk.ap-southeast-2.rds.amazonaws.com:5432/totago'
engine = create_engine(DATABASE_URI)

#engine = create_engine('sqlite:///totagoData.db')

# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)

session = DBSession()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


True
True


# Read in data as pandas data frame, selecting only certain fields

In [2]:
fields = ['distinct_id', 'numItinerariesReturned', 'departureDate', 'startFromLocation', 'selectedDestination_id', 'selectedDestination_name', 'time']

In [3]:
df = pd.read_csv('generated_itineraries.csv', usecols = fields)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Wrange field: destinationIDs

In [4]:
# Replace all of the NAs for destinationIDs with 0
df.selectedDestination_id.fillna(0, inplace = True)

# Remove the 2 cases where the string says null
# Great tutorial here: https://www.youtube.com/watch?v=2AFGPdNn4FM
df = df[df.selectedDestination_id != 'null']

# Convert destinationIDs column to an integer value
df['selectedDestination_id'] = df.selectedDestination_id.astype(int)



# Wrangle field: numItenerariesReturned

In [5]:
# Replace all of the NAs for numItinerariesReturned with 1
df.numItinerariesReturned.fillna(1, inplace = True)

# Convert from float to integer
df['numItinerariesReturned'] = df.numItinerariesReturned.astype(int)

# Select, only observatiosn where this field is greater than 0 (now that the NAs are gone)


# Wrangle Field: Destination Name

In [6]:
#Replace all of the NAs in
df.selectedDestination_name.fillna("", inplace = True)

print("Number of rows before departure date: ")
print(len(df))


Number of rows before departure date: 
28138


# Wrangle Field: departureDate

In [7]:
#Convert destinationIDs column to an integer value
# It looks like there were some complex rows being held in here before, I thought that when df.dtypes returned object that
# meant string but apprently not
df['departureDate'] = df.departureDate.astype(str)

print("number of rows before removal of anamoulous departureDate cases")
print(len(df))


# IT looks like there are some cases where this field is blank, says nan, is in format 24503, or in format "masked" 
# We need to remove these cases from the data frame
# I can see that some blank rows are still printed out.
df = df[df.departureDate != '']
df = df[df.departureDate != 'nan']
df = df[df.departureDate != '24503']
df = df[df.departureDate != '[masked]']


print("number of rows after removal of anamoulous departureDate cases")
print(len(df))

# Create a function extractDate that extracts the first ten characters of an input string
def extractDate(dateString):
    extractedDate = dateString[0:10]
    if len(extractedDate) < 10:
        print(extractedDate)
    return extractedDate

''' Code to test if the extractDate function works

# Apply this function to create  a new column
df['departureDateFixed'] = df.departureDate.apply(extractDate)

cols = ['distinct_id', 'departureDate', 'departureDateFixed', 'numItinerariesReturned', 'selectedDestination_id', 'selectedDestination_name', 'startFromLocation']

df = df[cols]
'''

# Override departure date extracting all of the null time stamps
df['departureDate'] = df.departureDate.apply(extractDate)


# Convert departure date into a time object in pandas
#See here: https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime
# Though this actually might not need to be done
#df['departureDate'] = df.departureDate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))




#df['departureDate'] = datetime.strptime(df['departureDate'], '%Y-%m-%-d')  
#df['departureDate'] = pd.to_datetime(df['departureDate'], format = '%Y-%m-%-d')



number of rows before removal of anamoulous departureDate cases
28138
number of rows after removal of anamoulous departureDate cases
28130


# Wrangle Field: distinctID

In [8]:
#It turns out distinc_id correpsonds to a user

# Therefore, create a coloumn that combines the unix time stamp with distinct_id so that we have a primary key for database
df["primary_key"] = df["distinct_id"] + "-" + df["time"].map(str)


vc = df.primary_key.value_counts()
print(vc[vc > 1])


unique_keys = df.primary_key.unique()
#print(len(unique_keys))

#df.head(n = len(df))

156c0427d8411b-089cb3f5e0f3f-1c114a5c-13c680-156c0427d85c5-1472086519       3
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989343                             3
16017f5f592536-094b655d7b8ce4-7636321b-4a640-16017f5f5932bd-1512207331      3
15ae497624281a-06e6a097b99f59-5e4f2b18-ff000-15ae4976243a16-1489889250      3
68f3348e-32ff-4756-a6b9-fbf722d5bf76-1460184823                             3
9a942f21-ad25-4129-b760-46bcec9e631d-1448989397                             2
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989357                             2
f763ca4a-e65f-4054-8a7a-8682bc627b15-1474711830                             2
1657f9a517fcd8-0eec0990ac17ad-49183707-13c680-1657f9a5180e51-1535419115     2
1618f2648b084b-06c2b14d540606-32657403-1aeaa0-1618f2648b147f-1518645763     2
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989351                             2
162f0142ee872-09409dc3356e2d-7047503f-144000-162f0142ee95ef-1524421918      2
16693ed1-fd53-4577-8cbe-5ca19ff74b89-1509430935                 

# Create a subset of the datle with sample method to test geocode and database entry logic

In [9]:
#Out put the entire database
#df.head(len(df))

len(df)

28130

In [10]:
#Create a random sample of the database, these entries will be added to the database in the next section
sampleDf = df.sample(1000)

# Output this random sample
sampleDf.head(len(sampleDf))    

,departureDate,distinct_id,numItinerariesReturned,selectedDestination_id,selectedDestination_name,startFromLocation,time,primary_key
24321,2018-07-20,164b0f3e48619-0d23f1fd0b1eb6-5b193413-100200-1...,1,131,Grouse Grind,"Seymour Street, Vancouver, BC, Canada",1532125706,164b0f3e48619-0d23f1fd0b1eb6-5b193413-100200-1...
8406,2017-02-11,e0ba25e3-4d8d-421e-ac7d-1912dc66e927,1,4,,My Location,1486806442,e0ba25e3-4d8d-421e-ac7d-1912dc66e927-1486806442
14404,2017-07-27,93f3dc47-30af-49a5-9644-7e84ee646eae,1,150,,My Location,1501126007,93f3dc47-30af-49a5-9644-7e84ee646eae-1501126007
3974,2016-08-14,1568735d79968f-01d0de1a7-683a296b-4a640-156873...,1,119,,"2465 West 6th Avenue, Vancouver, BC, Canada",1471122518,1568735d79968f-01d0de1a7-683a296b-4a640-156873...
23053,2018-06-24,1641a1097bf247-01101ae9d805a3-33657f07-13c680-...,1,146,Quarry Rock,"Newton Exchange Bay 1, Surrey, BC, Canada",1529420493,1641a1097bf247-01101ae9d805a3-33657f07-13c680-...
4903,2016-08-28,156d217b6548e5-0d977865a-504b7d3c-fa000-156d21...,1,145,,"6359 Agronomy Rd, Vancouver, BC, Canada",1472380006,156d217b6548e5-0d977865a-504b7d3c-fa000-156d21...
18769,2018-01-14,160f2071c961b5-0be4cc1f2878e5-163c6657-fa000-1...,1,46,Norvan Falls,"2033 Triumph Street, Vancouver, BC, Canada",1515860287,160f2071c961b5-0be4cc1f2878e5-163c6657-fa000-1...
6772,2016-10-22,157eb0d6bd42f7-087014a269df208-7a7a346b-3d10d-...,1,131,,"Metrotown, Burnaby, BC, Canada",1477092183,157eb0d6bd42f7-087014a269df208-7a7a346b-3d10d-...
21730,2018-05-18,163714b078c270-0231aba80541698-17347a40-100200...,1,148,Sasamat Lake,"Churchill Street, Vancouver, Britisch-Kolumbie...",1526589346,163714b078c270-0231aba80541698-17347a40-100200...
24671,2018-07-29,164e759e82f4ce-038742d298f4328-33780e30-3d10d-...,0,136,,"Vancouver City Centre Station, Granville Stree...",1532867596,164e759e82f4ce-038742d298f4328-33780e30-3d10d-...


## Read in the destination data to allow the possibility to pull the correct names

 


In [11]:
f = open("destinations_mapping_Jul-30-18.csv")

reader = csv.reader(f)


destinations = {}


# The index at the end of the for loop just skips the first row which is the header in the csv file
next(reader)
for row in reader:
    destinations[row[0]] = {'name':row[1]}

print(destinations)

{'168': {'name': 'Mammoth Pass - Crater Meadow Trail'}, '178': {'name': 'Sport Climbing at Horseshoe Slabs'}, '181': {'name': 'Hazel Wolf Wetlands Loop'}, '183': {'name': 'Redwood Grove Loop Trail'}, '184': {'name': 'Birdwatching at Fowlsheugh Reserve'}, '12': {'name': 'Point Defiance Loop (OLD)'}, '112': {'name': 'Baden Powell Lynn Canyon to Grouse'}, '175': {'name': 'Rainbow Falls Trail'}, '174': {'name': 'Walking the Town Loop to Sherwins Vista'}, '20': {'name': 'The Lions Binkert Trail'}, '27': {'name': 'Tumamoc Hill'}, '8': {'name': 'Proximity Alert Test'}, '169': {'name': 'McLeod Lake Spur'}, '165': {'name': 'Siskiyou Mountain (White Rabbit Trail)'}, '22': {'name': 'Lake Blanca'}, '11': {'name': 'Carkeek Park'}, '162': {'name': 'Rouge Park Traverse'}, '172': {'name': 'Walking the Lakes Basin Path at Twin Lakes'}, '16': {'name': 'Golden Gardens to Carkeek Park Beach Walk'}, '111': {'name': 'Baden Powell Deep Cove to Lynn Canyon'}, '278': {'name': 'Tujunga Wash Path'}, '13': {'name

# Loop through the rows in the dataframe, geocode, add entry to database

In [12]:
# Loop through the subsetted pandas data frame

# Uncomment the code below to loop through the the sample data frame
# for index, row in sampleDf.iterrows():

for index, row in sampleDf.iterrows():
  

    # Pull out the primary key into a variable
    testKey = row["primary_key"]
    
    # Check to see if that distinctID is in the data base
    # See this post: https://stackoverflow.com/questions/6587879/how-to-elegantly-check-the-existence-of-an-object-instance-variable-and-simultan?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
    entryExists = session.query(exists().where(Itenerary.distinctkey==testKey)).scalar()

    # If the entry is not in the database
    if not entryExists:
    
        # Get the string to be geocoded
        locationToGeocode = row["startFromLocation"]

        # Try to run the geocode function that returns a dictionary of information
        try:
            geocodeInfo = geocode(locationToGeocode)
            # If geocoding works, set valid to tre
            valid = True

        # If the geocode function doesn't work set valid to false
        except:
            valid = False 

        # If valid is true create a database entry with information from the dataframe, and the returned geocode informaiton
        if valid:
            
            # Sometimes, such as when a generic city is sent to the geocode() function a geometric center
            # is returned, this means there is no postal code
            
            # In this case, we can set the postalCode to One
            if not 'postalCode' in geocodeInfo:
                geocodeInfo['postalCode'] = "none"

            
            # Mapp the gps coordinates returned to the zip code polygons
            zipCodeMapped = mapToPoly(geocodeInfo['lat'], geocodeInfo['lng'], 'postal')
            
            barrioMapped = mapToPoly(geocodeInfo['lat'], geocodeInfo['lng'], 'barrio')
            
            ## Get selected Destination Names
            # Pull the selected destination name
            selectedDestinationName = row["selectedDestination_name"]
            
            if not selectedDestinationName:
                
                key = str(row["selectedDestination_id"])
                
                if key in destinations:
            
                    # Pull the data out from the dictionary that was created in the cell above
                    newName = destinations[str(row["selectedDestination_id"])]['name']
        
                    # Add the new name to the new row
                    selectedDestinationName  = newName
            
                # In the case that there is destination that corresponds mark
                else:
                
                    # TODO: CONSIDER CHANGING THE NAME OF THIS TO SOMETHING ELSE
                    selectedDestinationName = "DELETED"
                    # And overwrite valid to false at this point because there is no destination
                    valid = False
                
            databaseEntry = Itenerary(distinctkey=row["primary_key"],
                                      numberitinerariesreturned=row["numItinerariesReturned"],
                                      selecteddestination_id=row["selectedDestination_id"],
                                      selecteddestination_name=selectedDestinationName,
                                      startfromlocation=row["startFromLocation"],
                                      departuredate=row["departureDate"],
                                      # Get data from python dictionary returned from geocode() function
                                      formatted_address=geocodeInfo['formatted_address'],
                                      lat=geocodeInfo['lat'],
                                      lng=geocodeInfo['lng'],
                                      postalcode=geocodeInfo['postalCode'],
                                      postalcodemapped=zipCodeMapped,
                                      barriomapped=barrioMapped,
                                      valid=valid)
        # If valid is false, just fill in the information that we have from the pandas data frame
        else:
            databaseEntry = Itenerary(distinctkey=row["primary_key"],
                                      numberitinerariesreturned=row["numItinerariesReturned"],
                                      selecteddestination_id=row["selectedDestination_id"],
                                      selecteddestination_name=row["selectedDestination_name"],
                                      startfromlocation=row["startFromLocation"],
                                      departuredate=row["departureDate"],
                                      valid=valid)

        # Add the the information to a database.    
        session.add(databaseEntry)
        session.commit()
    
    else:
        print("Entry already inside database")

V6B
CBD
V5M
HS
V3W
V6T
V5L
GW
V5H
V6M
KERR
V7Y
CBD
98164
271849
98103
252248
V5K
HS
98121
271808
V6P
MARP
V6G
WE
98118
343994
V6J
SHAU
V6B
CBD
V6C
CBD
V5H
V3K
V6C
CBD
98105
272001
V7L
98101
271849
V3K
V5K
HS
V6C
CBD
V3L
98004
271850
V6Z
CBD
98102
250206
V3T
V5V
RP
V6C
CBD
V3M
98105
272001
V6T
V7V
V5Z
FAIR
V3T
V5X
SUN
V6P
MARP
V6G
WE
90028
32059
V5H
98105
272001
V3T
V6A
STR
V6K
KITS
V6T
V6B
CBD
V6B
CBD
98101
271849
V7M
V6Z
CBD
V5R
RC
V5C
V5H
V6B
CBD
V3J
98101
271849
V6G
WE
V6S
DS
V3M
V3R
V6C
CBD
V5V
KC
V5X
SUN
V6C
CBD
V6B
CBD
V6Y
V6C
CBD
V5T
MP
V6Z
CBD
V6A
CBD
V6E
WE
V6A
STR
V6E
CBD
V6J
SHAU
V6K
KITS
V5N
GW
90089
268540
V5N
KC
94706
264016
V6T
V5V
RP
V6C
CBD
V6B
CBD
V3N
V6H
FAIR
V7R
V5N
KC
V6B
CBD
V5P
VF
90038
32059
V5C
V5X
SUN
V7E
V8N
V7C
V5T
MP
V4K
V6G
CBD
V6T
98102
250206
90006
268404
V6A
CBD
V3T
V6B
CBD
V6A
STR
V6E
CBD
V6B
CBD
98102
271923
V5N
GW
V7E
V6T
98164
271849
V6B
CBD
98116
343997
98033
274599
V5V
RP
V6Y
V5C
V5Z
SC
V6E
WE
90068
274049
V5J
V6C
CBD
V6Y
V5E
V5Y
MP
V6E
WE
98126
3